In [2]:
# !pip install numpy
# !pip install pandas
# !pip install matplotlib
# !pip install scipy
# !pip install seaborn

In [54]:
import os
import numpy as np
import pandas as pd
from matplotlib import rc
import matplotlib.pyplot as plt
import matplotlib.font_manager as fm
import re
import glob
import seaborn as sns
from datetime import datetime, timedelta

In [3]:
os.getcwd()

'/Users/b05/Desktop/YEAR_DREAM/FC_PJT'

In [4]:
# Checking Fonts

rc('font', family='AppleGothic')
plt.rcParams['axes.unicode_minus'] = False
font_path = fm.findfont('AppleGothic')
if not font_path:
    print('Warning: AppleGothic font not found')
else:
    print("AppleGothic font found at ", font_path)

font_prop = fm.FontProperties(fname=font_path, size=12)

AppleGothic font found at  /System/Library/Fonts/Supplemental/AppleGothic.ttf


In [5]:
# Creating variables for data paths
dataset = "./dataset/"

In [58]:
if not os.path.exists(dataset + "Grouped"):
    os.mkdir(dataset + "Grouped")

In [6]:
df = pd.read_csv(dataset+'eda-proj-fc-purchase.csv')

In [7]:
df.rename(columns={"거래id":"transaction_id", "유형":"type", "고객id":"customer_id","코스ID":"courseID","사이트":"site", "포맷":"format", "카테고리":"category", "코스(상품) 이름":"course_name", "거래일자":"transaction_date_time","쿠폰이름":"coupon_name","판매가격":"sold_price","결제수단":"payment_method","실거래금액":"actual_sold_price","쿠폰할인액":"coupon_discount","거래금액":"transaction_amount","환불금액":"refund_amount"}, inplace=True)
df.to_csv(dataset + "rowdata_eng.csv", index=False)

In [8]:
rowdata_eng = pd.read_csv(dataset+"rowdata_eng.csv")

In [9]:
print(len(rowdata_eng[['transaction_date_time']]))
print(len(rowdata_eng))

159328
159328


In [10]:
df = rowdata_eng['transaction_date_time'].str.split(" ", n=3, expand=True)
print(df[3])
rowdata_eng['Date'] = df[0].str.cat([df[1],df[2]], sep="")
rowdata_eng['Time'] = df[3]

rowdata_eng.to_csv(dataset+"rowdata_eng_dtsplit.csv", index=False)

0         오후 10:12:33
1          오후 6:12:57
2         오후 11:12:15
3         오후 11:12:03
4         오후 11:12:10
             ...     
159323    오전 12:01:08
159324    오전 12:01:12
159325    오전 12:01:21
159326    오전 11:01:24
159327     오전 4:01:00
Name: 3, Length: 159328, dtype: object


In [26]:
rowdata_dtsplit = pd.read_csv(dataset+"rowdata_eng_dtsplit.csv")
tsplit = rowdata_dtsplit["Time"]
for i, time in tsplit.iteritems():
    if "오후" in time:
        time = time.replace("오후 ","")
        if int(time.split(":")[0]) == 12:
            pass
        else:
            x = int(time.split(":")[0]) + 12
            y = time.split(":")[1]
            z = time.split(":")[2]
            xyz = [str(x),":", str(y), ":", str(z)]
            time_str = "".join(xyz)
            rowdata_dtsplit.at[i, "Time"] = time_str
    else:
        time = time.replace("오전 ", "")
        rowdata_dtsplit.at[i, "Time"] = time

rowdata_dtsplit.to_csv(dataset+"24h time.csv",index=False)

/var/folders/0t/816d84452dn7h591gjkg32_m0000gn/T/ipykernel_35380/1254090522.py:3: FutureWarning: iteritems is deprecated and will be removed in a future version. Use .items instead.
  for i, time in tsplit.iteritems():


In [27]:
time_data = pd.read_csv(dataset+"24h time.csv")

In [56]:
def clean_course_data(course_name):
    if course_name.startswith("("):
        if "~" in course_name:
            return course_name.split(")")[1]
        else:
            return course_name
    else:
        return course_name
    
time_data["course_name"] = time_data["course_name"].apply(clean_course_data)
time_data.to_csv(dataset +"course_name_cleansed.csv", index = False)

In [65]:
def group_columns(row):
    parts = re.split("[:|-]", row["course_name"])
    if len(parts) == 1:
        return ("Other")
    return parts[0]

time_data["course_group"] = time_data.apply(group_columns, axis=1)
grouped_df = time_data.groupby("course_group")
for group_name, group_df in grouped_df:
    print("Group: ", group_name)
    groupname = group_name.replace("/","_")
    filename = f"{groupname.strip()}.csv"
    group_df.to_csv(dataset + f"Grouped/{filename}", index=False)
    

Group:   UX Explorer 
Group:   올인원 패키지 
Group:   초격차 패키지 
Group:  ((2022 리부트)) 초격차 패키지 
Group:  (2022 리뉴얼) 올인원 패키지 
Group:  3D 캐릭터를 가장 쉽게 만드는 법
Group:  AI/Data Science Conference 
Group:  CLASS
Group:  CLASS 
Group:  Kafka 완전 정복 
Group:  Kubernetes 자격증 마스터 클래스 
Group:  Live2D로 만드는 버츄얼 캐릭터 
Group:  Next.js 완전 정복 
Group:  Other
Group:  REC.ON 
Group:  THE RED 
Group:  The RED
Group:  The RED 
Group:  The Red 
Group:  The red 
Group:  [2022] 올인원 패키지 
Group:  [Kit] 닷밀의 미디어아트 그래픽 클래스 
Group:  [Kit] 입문자를 위한 풀스택 웹 개발 Kit 
Group:  [Kit]집에 있는 카메라로 시작하는 Photography 
Group:  sudo 
Group:  글로벌 엑스퍼트
Group:  글로벌 엑스퍼트 
Group:  김동원의 저예산 마케팅을 위한 검색 광고 최적화 
Group:  네오아카데미
Group:  네오아카데미 
Group:  닷밀 미디어아트 그래픽 클래스
Group:  매치무브 온보딩 
Group:  모두를 위한 앱 개발 
Group:  바이블 
Group:  부동산 디벨로퍼 아카데미 
Group:  사이드 프로젝트 
Group:  스프링러너의 스프링 아카데미
Group:  시그니처 패키지 
Group:  실무 초밀착 리눅스
Group:  실무 프로젝트로 배우는 Kotlin & Spring 
Group:  연습문제 패키지 
Group:  옥테인으로 만드는 뉴디멘션
Group:  올인원 패키지 
Group:  올인원패키지
Group:  올인원패키지 
Group:  재수강용
Gr